## Analizando la Fuerza de Ventas Tipo 1 (Grupal)
* Se eligió este cargo y este producto por ser el de mayo cantidad de colaboradores y por tener mayor rotacion historica desde el 2018

In [2]:
# Librerias Necesarias
import pandas as pd
import altair as alt
#import altair.vegalite.v4_8 as alt
import numpy as np

#from scipy.stats import zscore

### Definicion de CESE

In [3]:
# Extrae la base desde gsheet

def extrae_drive(url):
    edit = '/edit#gid='
    export ='/export?format=xlsx&gid='
    url = (url).replace(edit, export)
    return url

In [138]:
cesados = extrae_drive('https://docs.google.com/spreadsheets/d/1bO_T0lAQgjcpjFFNVAwiXd2obwRjJS3K/edit#gid=2050623625')
cesados = pd.read_excel(
        cesados,
        usecols='E,I,O,P,S,T,W,X',
        dtype = {'CODIGO':str} #Key debe ser en tipo texto
    ).rename(
        columns = { #Estandarizo los nombres para no tener problemas
            'Total o Temprana':'TIEMPO_CESE',
            'Producto':'PRODUCTO',
            'TIPO':'TIPO_CESE',
            'MOTIVO PRIMARIO':'MOTIVO_CESE'}
    ).query('CARGO == "VENTA GRUPAL" and CESE >= "2018-01-01"') #filtro solo el cargo que quiero

#### Conociendo a los Cesados

1. colaboadores con dos ceses, debido a tener re ingresos, se tomara en cuenta la ultima fecha de cese, lo otros seran descartados

In [139]:
#codigos unicos duplicados
#Se tomaran solo lo ultimos cess seun fecha
cesados.groupby(by = 'CODIGO', as_index=False).agg(n =('CODIGO','count')).query('n>=2')

,CODIGO,n
347,A33AAAA21A,2
381,A34AAAAA55,2
412,A36AAAAA18,2
699,A4AAAAA742,2
806,A57AAAAA17,2
815,A57AAAAA37,2
1151,A84AAAAA56,2
1170,A85AAAAA4A,2
1183,A86AAAAAA3,2
1390,AA7AAA2763,2


In [140]:
# Obtener solo ultimos cesados
cesados = cesados.sort_values(
                by = 'CESE',
                ascending = False # de mayor a menor
            ).drop_duplicates(
                'CODIGO',
                keep = 'first' # los mas recientes
            )

1. Utilizaré las fechas como inicio de mes para estandarizar la fecha de salida e ingreso con otras variables en le futuro
2. Hallando la antiguedad de Cese vs Ingreso en meses

In [141]:
cesados['CESE'] = cesados['CESE'].to_numpy().astype('datetime64[M]')
cesados['INGRESO'] = cesados['INGRESO'].to_numpy().astype('datetime64[M]')
# Halla la antiguedad de los cesados en meses
cesados['antiguedad_meses'] = (((cesados['CESE'] - cesados['INGRESO']).dt.days)/30).round(0) # Creo antiguedad

In [142]:
#reemplaznado los tipo de ces - por <6 meses
cesados.loc[cesados['TIEMPO_CESE'] == '-', 'TIEMPO_CESE'] = '>6 Meses'

In [116]:
tiempo = cesados.groupby(
            by = 'TIEMPO_CESE', as_index = False
        ).agg(ceses = ('CODIGO', 'count'))


alt.Chart(tiempo).mark_arc(innerRadius = 50).encode(
            theta = alt.Theta(field = 'ceses', type = 'quantitative'),
            color = alt.Color(
                    field = 'TIEMPO_CESE',
                    type = 'nominal',
                    legend = alt.Legend(title = "Tipo de Cese"),
                    scale=alt.Scale(domain = ['TEMPRANA', '>6 Meses'], range = ['#BFBFBF', '#347EBF']))
        ).properties(
            title = 'Ceses por Tiempo',
            width = 300,
            height = 200
        ).configure_view(
            strokeWidth = 0
        ).configure_axis(
            labelFontSize = 0,
            labelAngle = 0
        ).configure_title(
            fontSize = 20,
            font = 'Arial',
            color = '#03258C',
            anchor = 'middle') 

alt.Chart(...)

In [115]:
tipo = cesados.groupby(
            by = 'TIPO_CESE', as_index = False
        ).agg(ceses = ('CODIGO', 'count'))

alt.Chart(tipo).mark_arc(innerRadius = 50).encode(
            theta = alt.Theta(field = 'ceses', type = 'quantitative'),
            color = alt.Color(
                    field = 'TIPO_CESE',
                    type = 'nominal',
                    legend = alt.Legend(title = "Tipo de Cese"),
                    scale=alt.Scale(domain = ['INVOLUNTARIA','VOLUNTARIA'], range = ['#BFBFBF', '#347EBF']))
        ).properties(
            title = 'Ceses por Tipo',
            width = 300,
            height = 200
        ).configure_view(
            strokeWidth = 0
        ).configure_axis(
            labelFontSize = 0,
            labelAngle = 0
        ).configure_title(
            fontSize = 20,
            font = 'Arial',
            color = '#03258C',
            anchor = 'middle') 

alt.Chart(...)

In [117]:
motivos = cesados.groupby(
            by = 'MOTIVO_CESE', as_index = False
        ).agg(
            cesados = ('CODIGO', 'count'))

alt.Chart(motivos).mark_bar(color = '#347EBF').encode(
            y = alt.Y(
                'MOTIVO_CESE',
                sort = '-x',
                title = 'Motivo de Ceses'),
            x = alt.X(
                'cesados',
                title = None,
                axis = None)
        ).properties(
            title = 'Motivos de Cese más Frecuentes',
            width = 500,
            height = 400
        ).configure_view(
            strokeWidth = 0
        ).configure_axis(
            labelFontSize = 12,
            labelAngle = 0
        ).configure_title(
            fontSize = 20,
            font = 'Arial',
            color = '#03258C',
            anchor = 'start') 

alt.Chart(...)

In [79]:
cesados.antiguedad_meses.describe()

count    1621.000000
mean       18.515731
std        15.243636
min         0.000000
25%         7.000000
50%        14.000000
75%        25.000000
max        90.000000
Name: antiguedad_meses, dtype: float64

1. Principales motivos de cese involuntario, conlleva otro tipo de analisis por estar relacionado con recruting

In [89]:
cesados.query('TIPO_CESE == "INVOLUNTARIA"'
        ).groupby(
            by = 'MOTIVO_CESE', as_index = False
        ).agg(
            cesados = ('CODIGO', 'count')
        ).sort_values(by='cesados', ascending = False)

,MOTIVO_CESE,cesados
7,Fraude,119
10,Termino De Contrato,107
6,Faltas A La Metodología o Política,75
0,Abandono de Trabajo,51
3,Código de Ética,49
8,Programa de Acompañamiento de Mejora,43
2,Bajo desempeño,17
4,Defunción,6
1,Bajo Desempeño,1
5,Falsas Expectativas,1


1. Analizando solo los ceses voluntarios y la caracteristica de antiguedad que tiene cada uno
2. Como se vio en puntos anteriores la disribucion de las antiguedades no es normal y tiene una dipersion anta, se analizara con boxplot

In [122]:
motivo_cese = cesados.query('TIPO_CESE == "VOLUNTARIA"')

alt.Chart(motivo_cese).mark_boxplot(size = 20, extent  =0.2, color = '#347EBF').encode(
            y = alt.Y(
                'MOTIVO_CESE:N',
                title = None,),
            x = alt.X(
                'antiguedad_meses:Q',
                title = 'Antiguedad de Ceses (meses)')
        ).properties(
            title = 'Motivos de Cese más Frecuentes',
            width = 500,
            height = 400
        ).configure_view(
            strokeWidth = 0
        ).configure_axis(
            labelFontSize = 12,
            titleFontSize = 14,
            labelAngle = 0
        ).configure_title(
            fontSize = 20,
            font = 'Arial',
            color = '#03258C',
            anchor = 'start')

alt.Chart(...)

1. Investigando Personal Eventual y No le gusto el trabajo

In [ ]:
cesados.query('TIPO_CESE == "VOLUNTARIA" and antiguedad_meses >= 6 and MOTIVO_CESE=="No le Gusto el Trabajo"').describe()

In [133]:
motivos = cesados.query('TIPO_CESE == "VOLUNTARIA" and MOTIVO_CESE == "No le Gusto el Trabajo"'
    ).groupby(
        by = ['antiguedad_meses'],
        as_index = False
    ).agg(
        ceses = ('CODIGO', 'count')
    )
# GRAFICANDO
alt.Chart(motivos).mark_bar(color = '#347EBF').encode(
            x = alt.X(
                'antiguedad_meses',
                bin = alt.BinParams(step = 3),
                title = 'Antiguedad (meses)'),
            y = alt.Y(
                'ceses',
                title = 'Total Ceses')
        ).properties(
            title = 'Total de Ceses Segun Antiguedad - No le Gusto el Trabajo',
            width = 500,
            height = 300
        ).configure_view(
            strokeWidth = 0
        ).configure_axis(
            labelFontSize = 12,
            titleFontSize = 14,
            labelAngle = 0,
            grid = False
        ).configure_title(
            fontSize = 20,
            font = 'Arial',
            color = '#03258C',
            anchor = 'start')

alt.Chart(...)

In [135]:
cesados.query('TIPO_CESE == "VOLUNTARIA" and antiguedad_meses >= 6 and MOTIVO_CESE=="Personal Eventual"').describe()

,antiguedad_meses
count,4.000000
mean,7.000000
std,1.414214
min,6.000000
25%,6.000000
50%,6.500000
75%,7.500000
max,9.000000


1. Antiguedades de los ceses voluntariossgun motivos selecciondos a analizar

In [146]:
cesados.query('TIPO_CESE == "VOLUNTARIA" and MOTIVO_CESE != "Personal Eventual"'
                    ).groupby(
                        by = ['antiguedad_meses'],
                        as_index = False
                    ).agg(
                        ceses = ('CODIGO', 'count')
                    ).isnull().sum()

antiguedad_meses    0
ceses               0
dtype: int64

In [151]:
voluntarias = cesados.query('TIPO_CESE == "VOLUNTARIA" and MOTIVO_CESE != "Personal Eventual"'
                    ).groupby(
                        by = ['antiguedad_meses'],
                        as_index = False
                    ).agg(
                        ceses = ('CODIGO', 'count')
                    )
# GRAFICANDO
alt.Chart(voluntarias).mark_bar(color = '#347EBF').encode(
            x = alt.X(
                'antiguedad_meses',
                bin = alt.BinParams(step = 3),
                title = 'Antiguedad (meses)'),
            y = alt.Y(
                'ceses',
                title = 'Total Ceses'),
            color = alt.condition(
                alt.datum.antiguedad_meses < 9, 
                alt.value('#BFBFBF'),
                alt.value('#347EBF'))
        ).properties(
            title = 'Total de Ceses Voluntarios Segun Antiguedad',
            width = 500,
            height = 300
        ).configure_view(
            strokeWidth = 0
        ).configure_axis(
            labelFontSize = 12,
            titleFontSize = 14,
            labelAngle = 0,
            grid = False
        ).configure_title(
            fontSize = 20,
            font = 'Arial',
            color = '#03258C',
            anchor = 'start')

alt.Chart(...)

#### Conclusiones - Definicion de CESE

In [ ]:
cesados = cesados.query('TIPO_CESE == "VOLUNTARIA" and MOTIVO_CESE != "Personal Eventual" and antiguedad_meses >= 9')

### Analisis Base General

In [321]:
cesados.sample(10)

,CODIGO,CESE
2724,1A9AAAAA52,2020-12-01
2890,A34AAAAA8A,2021-03-01
1893,A41AAAAA31,2019-09-01
996,A46AAAAA15,2018-09-01
1366,A39AAAAAA4,2019-03-01
3458,AA7AAA4622,2021-12-01
2625,AA3AAAAAA1,2020-09-01
706,A55AAAAAA6,2018-03-01
2117,1AAAAAAA2A,2019-12-01
1939,AA7AAA4A62,2019-10-01


In [295]:
cesados = cesados[['CODIGO', 'CESE']]

#### Planilla historica

In [179]:
#HIstorico Totl de Colaboradores
planilla = extrae_drive('https://docs.google.com/spreadsheets/d/1L094YBwj3BvobbXb4t3ahzpmL6gBSxgs/edit#gid=648976010')
planilla = pd.read_excel(
                planilla,
                usecols='A,B,D'
            ).rename(
                columns={
                    'COD':'CODIGO'}
            ).query(
                'CARGO == "VENTA GRUPAL" and MES >= "2018-01-01"'
            ).drop_duplicates(
                'CODIGO',
                keep = 'last')

planilla = planilla['CODIGO'] #solo necesito los codigos hisotricos     

In [394]:
df = pd.merge(
            planilla,
            cesados,
            on = 'CODIGO',
            how = 'left'
        )

In [399]:
df.loc[df['CESE'].isna() == True, 'Estado'] = 'ACTIVO'
df.loc[df['CESE'].isna() == False, 'Estado'] = 'CESADO'
df.loc[df['Estado'] == 'ACTIVO', 'CESE'] = '2022-01-01'
df.loc[df['Estado'] == 'CESADO', 'CESE'] = df['CESE'] # no cesados con fecha de antiguedd al cierre

#### Datos Personales

In [403]:
datos_personales = extrae_drive('https://docs.google.com/spreadsheets/d/1ku5CfOhQBpML0gXkuRIIEDjqG0liPdKt/edit#gid=254224400')
datos_personales = pd.read_excel(
                  
                        datos_personales,
                        usecols = 'A, C:G'
                    ).sort_values(
                        by = 'INGRESO',
                        ascending = False
                    ).drop_duplicates(
                        'CODIGO',
                        keep = 'first')

datos_personales['INGRESO'] = datos_personales['INGRESO'].to_numpy().astype('datetime64[M]') # obtengoel mes inicial de ingreso

df = pd.merge(
        df,
        datos_personales,
        on = 'CODIGO',
        how = 'left')
        
# Hallando la antiguedad en la Fecha de cese y fecha ingreso
df['antiguedad'] = (((df['CESE'] - df['INGRESO']).dt.days)/30).round(0) #meses
df['edad'] = (((df['CESE'] - df['NACIMIENTO']).dt.days)/360).round(0) #anios

1. Hay colaboradores con antiguedad negativa, debido aque aluna vez cesaron,pero despues de un tiempo volviern a ingresar
2. Se les pondra la fecha de cese como de cierre
3. Procedemos a hallar la nuevaaniguedad con estos cambios

In [404]:
# Colaboradores con antiguedad errada
df.query('antiguedad < 0').sort_values(by = 'antiguedad')['CODIGO'].unique()
cod = df.query('antiguedad < 0')['CODIGO'].unique()
df.loc[df['CODIGO'].isin(cod), 'CESE'] = '2021-01-01'
df.loc[df['CODIGO'].isin(cod), 'Estado'] = 'ACTIVO'
df['antiguedad'] = (((df['CESE'] - df['INGRESO']).dt.days)/30).round(0) #meses

#### Bonos de Venta Variable

1. Analizo el comportamiento del RV en el tiempo
    * Como se ve, los periodos de marzo 2020 a octubre 2020 (inclusive) tienen valores muy bajos debido a la pandemia, por lo que para este analisis no lo tomaremos en cuenta, pues nos bajaria el promedio de maner atipica

In [405]:
bonos_venta = extrae_drive('https://docs.google.com/spreadsheets/d/1POTE8j5SneZ0QSvmbtKL_t4hu2odWlTM/edit#gid=578896078')
bonos_venta = pd.read_excel(
                    bonos_venta,
                    usecols = 'A, B, CE'
                ).rename(
                    columns = {
                    'Codigo Trabajador': 'CODIGO',
                    'RV TOTAL': 'RV'}
                )

In [406]:
# graficando rv historico incluido periodo covid
rv_agrup = bonos_venta.groupby(
                            by = 'MES',
                            as_index = False
                        ).agg(
                            rv_total = ('RV' , 'sum')
                        )

principal = alt.Chart(rv_agrup).mark_line(color = '#347EBF').encode(
                x = alt.X(
                    'MES:T',
                    title = None),
                y = alt.Y(
                    'rv_total:Q',
                    title = None,
                    axis = None)
            )

linea_i = alt.Chart(
                pd.DataFrame({'MES': ['2020-03-01']})
            ).mark_rule(
                color = '#F28729',
                strokeWidth = 2
            ).encode(x = 'MES:T')

linea_f = alt.Chart(
                pd.DataFrame({'MES': ['2020-10-01']})
            ).mark_rule(
                color = '#F28729',
                strokeWidth = 2
            ).encode(x = 'MES:T')

(principal + linea_i + linea_f).properties(
            title = 'RV Total Historico por Mes',
            width = 700,
            height = 300
        ).configure_view(
            strokeWidth = 0
        ).configure_axis(
            labelFontSize = 12,
            titleFontSize = 14,
            labelAngle = 0,
            grid = False
        ).configure_title(
            fontSize = 20,
            font = 'Arial',
            color = '#03258C',
            anchor = 'middle')

alt.LayerChart(...)

1. Agrupo los Rv por codigo obteniendo,rv mean, rv max y nro de veces que se obtuvo un rv de cero

In [407]:
bonos_venta = bonos_venta.query(
                'MES <= "2020-03-01" | MES >= "2020-10-01"') # filtro para no considerar covid

#bonos_venta = bonos_venta[['CODIGO','RV']] 
rvs = bonos_venta.groupby(by = 'CODIGO', as_index = False
                            ).agg(
                                rv_mean = ('RV' , 'mean'),
                                rv_max = ('RV' , 'max'))

rv_ceros = bonos_venta.query('RV <= 0').groupby(
                            by = 'CODIGO', as_index = False
                        ).agg(
                            rv_ceros = ('RV' , 'count'))

bonos_venta = pd.merge(
                rvs,
                rv_ceros,
                on = 'CODIGO',
                how = 'left')               

In [408]:
df = pd.merge(
        df,
        bonos_venta,
        on = 'CODIGO',
        how = 'left')

#### PRogram de mejora


1. Hallo el numero de veces que el colaborador estuvo en el programa de mejoras
    * Al parecer hay algunos colaboradores que tienen mas de 3 veces en el programa
    * El numero de veces que estuvo en el programa de mejora segun el motivo
2. Uno as bases del programa de mejora al df general

In [409]:
programa_mejora = extrae_drive('https://docs.google.com/spreadsheets/d/1CKFlLpx248yvZS0INAawAA5e8NC2qdvV/edit#gid=1510069928')
programa_mejora = pd.read_excel(
                    programa_mejora,
                    usecols = 'C, F, Q'
                ).rename(
                    columns = {
                    'CODIGO CFIS': 'CODIGO',
                    'MOTIVO PRINCIPAL': 'MOTIVO_PM'}
                ).query(
                    'Cargo == "VENTA GRUPAL"'
                )

#########################                
# Nro de veces en el programa
pm_total = programa_mejora.groupby(
                    by='CODIGO', as_index=False
                ).agg(
                    pm_total = ('CODIGO','count')
                )
                
# Nro de veces en el programa segun el motivo
pm_motivos = pd.get_dummies(
                programa_mejora,
                columns = ['MOTIVO_PM']
            ).groupby(
                by = 'CODIGO', as_index = False
            ).agg(
                MOTIVO_PM_DESEMP = ('MOTIVO_PM_DESEMPEÑO','sum'),
                MOTIVO_PM_DISCIPLINA = ('MOTIVO_PM_DISCIPLINA','sum')
            )
##########################
# Uniendo con total de programas
df = pd.merge(
        df,
        pm_total,
        on = 'CODIGO',
        how = 'left')

# Uniendo con motivos
df = pd.merge(
        df,
        pm_motivos,
        on = 'CODIGO',
        how = 'left')

#### Memos por Falta

In [410]:
memos = extrae_drive('https://docs.google.com/spreadsheets/d/1Q2pvICQWLAh8Rvou9FvH085KULXioDVK/edit#gid=238204385')
memos = pd.read_excel(
                    memos,
                    usecols = 'B, D'
                ).rename(
                    columns = {
                    'CÓDIGO': 'CODIGO'}
                ).query(
                    'CARGO == "VENTA GRUPAL"'
                )
# Memor por colaborador
memos = memos.groupby(
                by = 'CODIGO',
                as_index = False
            ).agg(
                memos_total = ('CODIGO','count')
            )

# uniendo total de memos al df             
df = pd.merge(
        df,
        memos,
        on = 'CODIGO',
        how = 'left')

#### Total de Hijos
1. Hallando la edad del hijx a la fecha de cese
    * Solo tomare en cuenta a los hijxs con edad menor o igual a 18 en la fecha de cese
    * Para esto necesito optener la fecha de cese y hallar la edad
2. Filtrar la edad menor a 18 y contar el numero de hijxs
3. Unir la cantidad de hijxs con la base df 

In [411]:
hijxs = extrae_drive('https://docs.google.com/spreadsheets/d/16lmjxH0f9sav7thbqeWGrdEsckr9Ia_u/edit#gid=1468377625')
hijxs = pd.read_excel(
                    hijxs,
                    usecols = 'D, F'
                ).rename(
                    columns = {'COD CFIS': 'CODIGO'}
                )
                        
# optener la fecha de cese
hijxs = pd.merge(
        hijxs,
        df[['CODIGO','CESE']],
        on = 'CODIGO',
        how = 'left')

# Halla la edad en el cese
hijxs['edad'] = (((hijxs['CESE'] - hijxs['FECHA_NACIMIENTO']).dt.days)/360).round(0)

# Total hijos menores a 18
hijxs = hijxs.query(
                'edad <= 18'
            ).groupby(
                by = 'CODIGO',
                as_index = False
            ).agg(
                hijxs = ('CODIGO','count')     
            )
            
# Uniendo hijos con df
df = pd.merge(
        df,
        hijxs,
        on = 'CODIGO',
        how = 'left')

In [412]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3117 entries, 0 to 3116
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   CODIGO                3117 non-null   object        
 1   CESE                  3117 non-null   datetime64[ns]
 2   Estado                3117 non-null   object        
 3   INGRESO               3117 non-null   datetime64[ns]
 4   NACIMIENTO            3117 non-null   datetime64[ns]
 5   SEXO                  3117 non-null   object        
 6   Educacion             3117 non-null   object        
 7   EC                    3117 non-null   object        
 8   antiguedad            3117 non-null   float64       
 9   edad                  3117 non-null   float64       
 10  rv_mean               3056 non-null   float64       
 11  rv_max                3056 non-null   float64       
 12  rv_ceros              2485 non-null   float64       
 13  pm_total          

In [431]:
rv_faltantes = df.groupby(by='antiguedad', as_index=False
                ).agg(
                    rv_prom_f = ('rv_mean', 'mean'),
                    rv_maximo_f = ('rv_max', 'mean'))

df = pd.merge(
        df,
        rv_faltantes,
        on = 'antiguedad',
        how = 'left')


In [432]:
df.loc[df['rv_mean'].isna() == True, 'rv_mean'] = df['rv_prom_f']
df.loc[df['rv_mean'].isna() == False, 'rv_mean'] = df['rv_mean']
df.loc[df['rv_max'].isna() == True, 'rv_max'] = df['rv_maximo_f']
df.loc[df['rv_max'].isna() == False, 'rv_max'] = df['rv_max']

In [434]:
# Reemplazando con 0 los colaboradores sin RV en CERo
df['rv_ceros'] = df['rv_ceros'].fillna(0)

# Reemplazando pm_total y motivos del programa de mejora vacios por 0
df['pm_total'] = df['pm_total'].fillna(0)
df['MOTIVO_PM_DESEMP'] = df['MOTIVO_PM_DESEMP'].fillna(0)
df['MOTIVO_PM_DISCIPLINA'] = df['MOTIVO_PM_DISCIPLINA'].fillna(0)

# Reemplazando con 0 los memos vacios
df['memos_total'] = df['memos_total'].fillna(0)

# Reemplazando con 0 los colaboradores sin hijxs
df['hijxs'] = df['hijxs'].fillna(0)


In [435]:
# Estandarizando Estado civil
df.loc[df['EC']=='CONCUBINATO / CONVIVIENTE', 'EC'] = 'CASADO'
df.loc[df['EC']=='SEPARADO', 'EC'] = 'DIVORCIADO'
df.loc[df['EC']=='SOLTERO/A', 'EC'] = 'SOLTERO'

# Estandarizando Educacion
df.loc[df['Educacion']=='EDUCACIÓN UNIVERSITARIA COMPLETA', 'Educacion'] = 'Univ comp'
df.loc[df['Educacion']=='EDUCACIÓN UNIVERSITARIA INCOMPLETA', 'Educacion'] = 'Univ inc'
df.loc[df['Educacion']=='GRADO DE BACHILLER', 'Educacion'] = 'Univ comp'
df.loc[df['Educacion']=='TITULADO', 'Educacion'] = 'Univ comp'
df.loc[df['Educacion']=='EDUCACIÓN TÉCNICA INCOMPLETA', 'Educacion'] = 'Tec inc'
df.loc[df['Educacion']=='EDUCACIÓN TÉCNICA COMPLETA', 'Educacion'] = 'Tec comp'
df.loc[df['Educacion']=='EDUCACIÓN SUPERIOR (INSTITUTO SUPERIOR, ETC) INCOMPLETA', 'Educacion'] = 'Tec inc'
df.loc[df['Educacion']=='EDUCACIÓN SUPERIOR (INSTITUTO SUPERIOR, ETC) COMPLETA', 'Educacion'] = 'Tec comp'
df.loc[df['Educacion']=='ESTUDIOS DE MAESTRÍA INCOMPLETA', 'Educacion'] = 'Maestria'
df.loc[df['Educacion']=='EDUCACIÓN SECUNDARIA COMPLETA', 'Educacion'] = 'Sec'

In [453]:
df_f = df[['CODIGO','Estado','SEXO','Educacion','EC','antiguedad','edad','rv_mean','rv_max','rv_ceros','pm_total',
    'MOTIVO_PM_DESEMP','MOTIVO_PM_DISCIPLINA','memos_total','hijxs']]

In [436]:
df.isna().sum()

CODIGO                  0
CESE                    0
Estado                  0
INGRESO                 0
NACIMIENTO              0
SEXO                    0
Educacion               0
EC                      0
antiguedad              0
edad                    0
rv_mean                 0
rv_max                  0
rv_ceros                0
pm_total                0
MOTIVO_PM_DESEMP        0
MOTIVO_PM_DISCIPLINA    0
memos_total             0
hijxs                   0
rv_prom_f               0
rv_maximo_f             0
dtype: int64

In [439]:
# rv promedio
alt.Chart(df).mark_boxplot(size=50, extent=0.5).encode(
        y = alt.X(
            'Estado:O',
            title = None),
        x = alt.Y(
            'antiguedad:Q',
            scale = alt.Scale(zero=False),
            title = 'Bono Variable Promedio'),
        ).properties(
            title = 'Rv Promedio Cesados vs Activos',
            width = 300,
            height = 300
        )

alt.Chart(...)

In [443]:
antg_estado=df.groupby(by=['antiguedad','Estado'], as_index=False).agg(n=('CODIGO','count'))

In [447]:
alt.Chart(antg_estado
).mark_bar(
    opacity=0.3,
    binSpacing=0
).encode(
    alt.X('antiguedad', bin=alt.BinParams(step = 3)),
    alt.Y('n', stack=None),
    alt.Color('Estado')
)

alt.Chart(...)

In [448]:
import sweetviz

In [456]:
df_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3117 entries, 0 to 3116
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   CODIGO                3117 non-null   object 
 1   antiguedad            3117 non-null   float64
 2   edad                  3117 non-null   float64
 3   rv_mean               3117 non-null   float64
 4   rv_max                3117 non-null   float64
 5   rv_ceros              3117 non-null   float64
 6   pm_total              3117 non-null   float64
 7   MOTIVO_PM_DESEMP      3117 non-null   float64
 8   MOTIVO_PM_DISCIPLINA  3117 non-null   float64
 9   memos_total           3117 non-null   float64
 10  hijxs                 3117 non-null   float64
 11  SEXO_M                3117 non-null   uint8  
 12  SEXO_f                3117 non-null   uint8  
 13  Estado_CESADO         3117 non-null   uint8  
 14  EC_CASADO             3117 non-null   uint8  
 15  EC_DIVORCIADO        

In [461]:
 pd.get_dummies(df, columns = ['SEXO'], drop_first=True)

,CODIGO,CESE,Estado,INGRESO,NACIMIENTO,Educacion,EC,antiguedad,edad,rv_mean,...,rv_ceros,pm_total,MOTIVO_PM_DESEMP,MOTIVO_PM_DISCIPLINA,memos_total,hijxs,rv_prom_f,rv_maximo_f,SEXO_M,SEXO_f
0,AA7AAA3426,2018-02-01,CESADO,2014-03-01,1989-07-31,Univ comp,SOLTERO,48.0,29.0,264.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,4258.684544,15198.497209,1,0
1,AA7AAA4153,2018-02-01,CESADO,2015-08-01,1994-09-12,Univ inc,CASADO,30.0,24.0,523.080000,...,0.0,0.0,0.0,0.0,1.0,0.0,4157.480340,12446.558138,1,0
2,AA8AAA2391,2018-02-01,CESADO,2015-09-01,1988-07-08,Univ comp,SOLTERO,29.0,30.0,303.200000,...,0.0,1.0,1.0,0.0,1.0,0.0,3361.102116,10287.869022,1,0
3,AA7AAA4557,2022-01-01,ACTIVO,2016-01-01,1990-01-16,Univ comp,SOLTERO,73.0,32.0,6088.986109,...,0.0,0.0,0.0,0.0,1.0,2.0,6088.986109,15924.300388,1,0
4,A4AAAAA287,2022-01-01,ACTIVO,2016-02-01,1993-05-12,Univ comp,SOLTERO,72.0,29.0,108.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,4223.741386,14910.511105,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3112,A99AAAAA47,2022-01-01,ACTIVO,2021-12-01,1990-08-08,Tec comp,SOLTERO,1.0,32.0,1740.485714,...,0.0,0.0,0.0,0.0,0.0,1.0,1740.485714,1740.485714,0,0
3113,A49AAAA116,2022-01-01,ACTIVO,2021-12-01,1991-11-13,Univ comp,CASADO,1.0,31.0,1740.485714,...,0.0,0.0,0.0,0.0,0.0,1.0,1740.485714,1740.485714,0,0
3114,A22AAAAA77,2022-01-01,ACTIVO,2021-12-01,1999-05-03,Tec comp,SOLTERO,1.0,23.0,1740.485714,...,0.0,0.0,0.0,0.0,0.0,0.0,1740.485714,1740.485714,0,0
3115,A52AAAAA63,2022-01-01,ACTIVO,2021-12-01,1996-04-09,Tec comp,CASADO,1.0,26.0,100.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,1740.485714,1740.485714,0,0


In [455]:
df_f = pd.get_dummies(df_f, columns = ['SEXO', 'Estado'], drop_first=True)

# Tranformando Estado Civil y Educacion
df_f = pd.get_dummies(df_f, columns = ['EC', 'Educacion'])

In [459]:
my_report  = sweetviz.analyze([df_f,'CODIGO' ], target_feat='Estado_CESADO')
my_report.show_html('FinalReport.html')

Feature: Estado (TARGET)                     |▉         | [ 10%]   05:58 -> (56:44 left)
Feature: Estado (TARGET)                     |▉         | [ 10%]   00:52 -> (08:15 left)
Done! Use 'show' commands to display/save.   |██████████| [100%]   00:02 -> (00:00 left)


Report FinalReport.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
